# Step 1: Create twitter connection

In [1]:
#It creates the Twitter API connection and the topic (covidtweets) to pass the information.
!curl -X POST -H "Content-Type: application/json" --data @configs/twitterConnector.json http://connect:8083/connectors | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1587  100   781  100   806   9409   9710 --:--:-- --:--:-- --:--:-- 19120
{
   "config" : {
      "connector.class" : "com.github.jcustenborder.kafka.connect.twitter.TwitterSourceConnector",
      "filter.keywords" : "covid, covid-19, vaccine",
      "kafka.status.topic" : "covidtweets",
      "name" : "TwitterConnector",
      "process.deletes" : "true",
      "quickstart" : "covidtweets",
      "tasks.max" : "1",
      "topic.creation.default.partitions" : "1",
      "topic.creation.default.replication.factor" : "1",
      "topic.creation.groups" : "covidtweets",
      "twitter.debug" : "true",
      "twitter.oauth.accessToken" : "1399325642294927368-K1AJ4EDkX9FwsBcDDcd20FT5aZTINH",
      "twitter.oauth.accessTokenSecret" : "E1w66uA9xHdserUMXhuPyYMLT9A68qUXzLNccVG4u4CEd",
      "twitter.oauth.consumerKey" : "8RcSxDLeBIldxfM

# Step 2: Create streams (ksql)

In [2]:
# It creates the data stream form the covidtweets topic. It is called RAWTWEETS
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/RawStream.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3148  100  3026  100   122   6940    279 --:--:-- --:--:-- --:--:--  7203
[
   {
      "@type" : "currentStatus",
      "commandId" : "stream/`RAWTWEETS`/create",
      "commandSequenceNumber" : 2,
      "commandStatus" : {
         "message" : "Stream created",
         "queryId" : null,
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE STREAM RAWTWEETS (CREATEDAT BIGINT, ID BIGINT, TEXT STRING, SOURCE STRING, TRUNCATED BOOLEAN, INREPLYTOSTATUSID BIGINT, INREPLYTOUSERID BIGINT, INREPLYTOSCREENNAME STRING, GEOLOCATION STRUCT<LATITUDE DOUBLE, LONGITUDE DOUBLE>, PLACE STRUCT<NAME STRING, STREETADDRESS STRING, COUNTRYCODE STRING, ID STRING, COUNTRY STRING, PLACETYPE STRING, URL STRING, FULLNAME STRING>, FAVORITED BOOLEAN, RETWEETED BOOLEAN, FAVORITECOUNT INTEGER, USER STRUCT<ID BIGINT, NAME STRING, SCREENNAM

In [3]:
# It creates a  stream from the RAWTWEETS. It selects some of the features and filters lang = eng. It is called ENGSTREAM
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/EngStream.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2007  100  1239  100   768   1002    621  0:00:01  0:00:01 --:--:--  1623
[
   {
      "@type" : "currentStatus",
      "commandId" : "stream/`ENGSTREAM`/create",
      "commandSequenceNumber" : 4,
      "commandStatus" : {
         "message" : "Created query with ID CSAS_ENGSTREAM_3",
         "queryId" : "CSAS_ENGSTREAM_3",
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE STREAM ENGSTREAM WITH (KAFKA_TOPIC='engtweets', PARTITIONS=1, REPLICAS=1, VALUE_FORMAT='JSON') AS SELECT\n  RAWTWEETS.CREATEDAT CREATEDAT,\n  RAWTWEETS.ID ID,\n  RAWTWEETS.USER->ID USER_ID,\n  RAWTWEETS.USER->NAME USER_NAME,\n  RAWTWEETS.USER->SCREENNAME SCREENNAME,\n  RAWTWEETS.USER->LOCATION USER_LOCATION,\n  RAWTWEETS.USER->FOLLOWERSCOUNT FOLLOW_COUNT,\n  RAWTWEETS.USER->FRIENDSCOUNT FRIEND_COUNT,\n  RAWTWEETS.USER->CREATEDAT USER_

In [8]:
# It creates a stream from the RAWTWEETS. It selects some of the features and filters country is not null. It is called LOCSTREAM
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/LocStream.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1989  100  1223  100   766   2412   1510 --:--:-- --:--:-- --:--:--  3923
[
   {
      "@type" : "currentStatus",
      "commandId" : "stream/`LOCSTREAM`/create",
      "commandSequenceNumber" : 18,
      "commandStatus" : {
         "message" : "Created query with ID CSAS_LOCSTREAM_17",
         "queryId" : "CSAS_LOCSTREAM_17",
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE STREAM LOCSTREAM WITH (KAFKA_TOPIC='loctweets', PARTITIONS=1, REPLICAS=1, VALUE_FORMAT='JSON') AS SELECT\n  RAWTWEETS.CREATEDAT CREATEDAT,\n  RAWTWEETS.ID ID,\n  RAWTWEETS.USER->ID USER_ID,\n  RAWTWEETS.USER->NAME USER_NAME,\n  RAWTWEETS.USER->SCREENNAME SCREENNAME,\n  RAWTWEETS.USER->LOCATION USER_LOCATION,\n  RAWTWEETS.USER->FOLLOWERSCOUNT FOLLOW_COUNT,\n  RAWTWEETS.USER->FRIENDSCOUNT FRIEND_COUNT,\n  RAWTWEETS.USER->CREATEDAT US

In [5]:
# It creates a stream from the RAWTWEETS. It lists the Hashtags on the tweets. It is called HASHSTREAM
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/HashStream.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   649  100   458  100   191   1221    509 --:--:-- --:--:-- --:--:--  1726
[
   {
      "@type" : "currentStatus",
      "commandId" : "stream/`HASHSTREAM`/create",
      "commandSequenceNumber" : 8,
      "commandStatus" : {
         "message" : "Created query with ID CSAS_HASHSTREAM_7",
         "queryId" : "CSAS_HASHSTREAM_7",
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE STREAM HASHSTREAM WITH (KAFKA_TOPIC='HASHSTREAM', PARTITIONS=1, REPLICAS=1) AS SELECT\n  RAWTWEETS.ID ID,\n  LCASE(EXPLODE(RAWTWEETS.HASHTAGENTITIES)->TEXT) HASHTAG\nFROM RAWTWEETS RAWTWEETS\nEMIT CHANGES;",
      "warnings" : []
   }
]


# Step 3: Create tables (ksql)

In [6]:
# It creates a table from HASHSTREAM which count the tweets per hashtag in a 10 min window (TUMBLING)
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/HashTable.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   655  100   488  100   167   1053    360 --:--:-- --:--:-- --:--:--  1414
[
   {
      "@type" : "currentStatus",
      "commandId" : "table/`HASHCOUNT`/create",
      "commandSequenceNumber" : 10,
      "commandStatus" : {
         "message" : "Created query with ID CTAS_HASHCOUNT_9",
         "queryId" : "CTAS_HASHCOUNT_9",
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE TABLE HASHCOUNT WITH (KAFKA_TOPIC='HASHCOUNT', PARTITIONS=1, REPLICAS=1) AS SELECT\n  HASHSTREAM.HASHTAG HASHTAG,\n  COUNT(*) CT\nFROM HASHSTREAM HASHSTREAM\nWINDOW TUMBLING ( SIZE 10 MINUTES ) \nGROUP BY HASHSTREAM.HASHTAG\nEMIT CHANGES;",
      "warnings" : []
   }
]


In [7]:
# It creates a table from RAWTWEETS which count the tweets per verified account.
!curl -X POST  -H "Content-Type: application/json; charset=utf-8" -d @ksql/VerTable.json http://ksqldb-server:8088/ksql | json_pp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   846  100   603  100   243   1518    612 --:--:-- --:--:-- --:--:--  2125
[
   {
      "@type" : "currentStatus",
      "commandId" : "table/`COUNT_USER_VERIFIED`/create",
      "commandSequenceNumber" : 12,
      "commandStatus" : {
         "message" : "Created query with ID CTAS_COUNT_USER_VERIFIED_11",
         "queryId" : "CTAS_COUNT_USER_VERIFIED_11",
         "status" : "SUCCESS"
      },
      "statementText" : "CREATE TABLE COUNT_USER_VERIFIED WITH (KAFKA_TOPIC='COUNT_USER_VERIFIED', PARTITIONS=1, REPLICAS=1) AS SELECT\n  RAWTWEETS.USER->SCREENNAME USER_SCREENNAME,\n  COUNT(*) COUNT_TWEETS\nFROM RAWTWEETS RAWTWEETS\nWINDOW TUMBLING ( SIZE 5 MINUTES ) \nWHERE RAWTWEETS.USER->VERIFIED\nGROUP BY RAWTWEETS.USER->SCREENNAME\nEMIT CHANGES;",
      "warnings" : []
   }
]
